In [174]:
import pandas as pd
import numpy as np
import os

In [175]:
# creating the dataframes
schoolsdf = pd.read_csv(os.path.join("raw_data","schools_complete.csv"))
studentsdf =  pd.read_csv(os.path.join("raw_data","students_complete.csv"))
schoolsdf.describe()


,School ID,size,budget
count,15.000000,15.000000,1.500000e+01
mean,7.000000,2611.333333,1.643295e+06
std,4.472136,1420.915282,9.347763e+05
min,0.000000,427.000000,2.480870e+05
25%,3.500000,1698.000000,1.046265e+06
50%,7.000000,2283.000000,1.319574e+06
75%,10.500000,3474.000000,2.228999e+06
max,14.000000,4976.000000,3.124928e+06


In [183]:
# This cell is for learning about the Schools Data
#schoolsdf.describe()

schoolsdf = schoolsdf.rename(columns = {'name':'school'})
schoolsdf

,School ID,school,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [177]:
#This cell is for learningabout the Students Data
#studentsdf.describe()
studentsdf.columns

Index(['Student ID', 'name', 'gender', 'grade', 'school', 'reading_score',
       'math_score'],
      dtype='object')

In [178]:
studentsdf.describe()


,Student ID,reading_score,math_score
count,39170.000000,39170.00000,39170.000000
mean,19584.500000,81.87784,78.985371
std,11307.549359,10.23958,12.309968
min,0.000000,63.00000,55.000000
25%,9792.250000,73.00000,69.000000
50%,19584.500000,82.00000,79.000000
75%,29376.750000,91.00000,89.000000
max,39169.000000,99.00000,99.000000


In [179]:
# District Summary
TotalSchools = schoolsdf['School ID'].count()
TotalStudents = studentsdf['Student ID'].count()
Totalbudget = schoolsdf['budget'].sum()
AvgMathScore = studentsdf['math_score'].mean()
AvgReadingScore = studentsdf['reading_score'].mean()
PerMathPass= (len(studentsdf.loc[studentsdf['math_score'] >65])/TotalStudents)*100
PerReadingPass = (len(studentsdf.loc[studentsdf['reading_score'] >65])/TotalStudents)*100
OverallPassingRate =(PerMathPass+PerReadingPass)/2


DistrictSummary= {'Total Schools': TotalSchools,'Total Students' : TotalStudents,'Total budget': Totalbudget,
                  'Average Math Score': AvgMathScore, 'Average Reading Score':AvgReadingScore,
                  '% Passing Math': PerMathPass, '% Passing Reading' :PerReadingPass, 'Overall Passing Rate' : OverallPassingRate}

DistrictSummaryDf = pd.DataFrame(DistrictSummary,columns = DistrictSummary.keys(),index=[0])
DistrictSummaryDf

,Total Schools,Total Students,Total budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,83.112076,94.263467,88.687771


In [213]:
#School Summary
#Final columns in the report
SummaryCol = ['School Name','School Type','Total Students','Total School Budget',
              'Per Student Budget','Average Math Score','Average Reading Score',
              '% Passing Math','% Passing Reading', 'Overall Passing Rate']


# We are creating this report in 4 steps.
#Step 1 : Group by the students dataframe by school and aggregate as many fields as you ca
schoolSumDf = studentsdf.groupby("school").agg({'reading_score':['mean'],
                                                'math_score':['mean']})
schoolSumDf.reset_index(inplace = True)
schoolSumDf.columns=[''.join(col) for col in schoolSumDf.columns ]
schoolSumDf = schoolSumDf.rename(columns = {'reading_scoremean': 'Average Reading Score','math_scoremean': 'Average Math Score'})


#Step 2 : Create 2 dataframes with number of students passing math and reading
StudentMathPassDf = studentsdf.loc[studentsdf['math_score'] >65,['school','math_score']]
StudentMathPassDf=StudentMathPassDf.groupby("school").count()
#StudentMathPassDf.reset_index(inplace = True)
StudentMathPassDf.reset_index(inplace =True)
pd.DataFrame(StudentMathPassDf.to_records()) 
StudentMathPassDf = StudentMathPassDf.rename(columns = {'math_score': 'NumMathPassed'})

StudentReadingPassDf = studentsdf.loc[studentsdf['reading_score'] >65,['school','reading_score']]
StudentReadingPassDf=StudentReadingPassDf.groupby("school").count()
StudentReadingPassDf.reset_index(inplace =True)
pd.DataFrame(StudentReadingPassDf.to_records())
StudentReadingPassDf = StudentReadingPassDf.rename(columns = {'reading_score': "NumReadingPassed"})
                                        
#Step 3 combine the schoolsDf,SchoolSumDf,StudentMathPassDf,StudentReadingPassDf 
merdf1 = pd.merge(schoolsdf,schoolSumDf,how = 'outer', on='school')
merdf1.reset_index()

merdf2 = pd.merge(StudentMathPassDf,StudentReadingPassDf,on='school')
merdf2.reset_index()

merdf3 = pd.merge(merdf1,merdf2,on='school')
merdf3.reset_index()
FinalDf = merdf3
FinalDf['Per Student Budget'] = FinalDf['budget']/FinalDf['size']
FinalDf['% Passing Math'] = (FinalDf['NumMathPassed']/FinalDf['size'])*100
FinalDf['% Passing Reading'] = (FinalDf['NumReadingPassed']/FinalDf['size'])*100
FinalDf['Overall Passing Rate'] = (((FinalDf['NumReadingPassed']/FinalDf['size'])*100)+
                                ((FinalDf['NumMathPassed']/FinalDf['size'])*100))/2
FinalDf['Per Student Budget'] = FinalDf['budget']/FinalDf['size']

ReportDf = FinalDf.rename(columns = {'school': 'School Name','type': 'School Type','size' :'Total Students','budget':'Total School Budget'})
ReportDf1 = ReportDf[SummaryCol]
ReportDf1


,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,75.282825,91.772369,83.527597
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,74.872838,91.895558,83.384198
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,100.000000,100.000000,100.000000
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,75.469256,91.456311,83.462783
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,100.000000,100.000000,100.000000
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,100.000000,100.000000,100.000000
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,100.000000,100.000000,100.000000
7,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,75.602894,91.901125,83.752010
8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,100.000000,100.000000,100.000000
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,100.000000,100.000000,100.000000
